# Naive Bayes Classifier with Kernel Density Estimators

The following code trains a Naive Bayes Classifier with KDEs to predict the distance between the Raspberrry Pis. The code below is adapted from Chapter 5 of Jake VanderPlas' *Python Data Science Handbook* and utilizes the `KDECLassifier` class from the same source.

In [ ]:
from kde_classifier import KDEClassifier
import pandas as pd
from pathlib import Path
from pi_pact_sort import categorize
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

DROP_COLUMNS = ['ADDRESS', 'TIMESTAMP', 'UUID', 'MAJOR', 'MINOR', 'TX POWER', 'TEMPERATURE',
                'PITCH', 'ROLL', 'YAW', 'SCAN']
SAMPLE_SIZE = 30000



"""Creates a Naive Bayes classifier with KDE to predict a distance range given RSSI values and other variables.

   KDEClassifier class used from Chapter 5 of the Python Data Science Handbook by Jake VanderPlas:
   https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html
"""

# Initialize DataFrame
data: pd.DataFrame = pd.DataFrame(columns=['RSSI', 'DISTANCE', 'HUMIDITY', 'PRESSURE'])
data_copy: pd.DataFrame = data.copy()
csv_file: Path
for csv_file in Path('.').glob('indoor-noObstruct-SenseHat*/*.csv'):
    datapart: pd.DataFrame = pd.read_csv(csv_file)
    for column in DROP_COLUMNS:
        if column in datapart.columns:
            datapart = datapart.drop([column], 1)
    data_copy = data_copy.append(datapart)

# Categorize distance
data_copy['DISTANCE'] = data_copy['DISTANCE'].map(categorize)

# Sample data from each distance category
for value in data_copy['DISTANCE'].unique():
    datapart = data_copy[data_copy.DISTANCE == value]
    datapart = datapart.sample(SAMPLE_SIZE)
    data = data.append(datapart)

# Assign features and labels
min_max_scaler = MinMaxScaler()
X: np.array = min_max_scaler.fit_transform(data.drop(['DISTANCE'], 1).to_numpy())
y: np.array = data['DISTANCE'].to_numpy(dtype=int)

# Hyperparameter tuning
# Code adapted from Chapter 5 of the Python Data Science Handbook by Jake VanderPlas:
# https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html
bandwidths = np.around(10 ** np.linspace(0, 2, 10), decimals=4)
grid = GridSearchCV(KDEClassifier(), {'bandwidth': bandwidths}, n_jobs=1)
grid.fit(X, y)

In [ ]:
print(grid.best_params_)
print('accuracy =', grid.best_score_)